# Imports and setting up libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# importing necessary libraries
import os
from tqdm.auto import tqdm
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import transforms

In [ ]:
!pip install torchinfo
from torchinfo import summary

In [ ]:
os.chdir('/content/gdrive/MyDrive/Deep Learning/pytorch-experiments/Vision/CIFAR-10')

In [ ]:
# using GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# ensuring reproducibility of code
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Loading data

In [ ]:
# defining transforms and data augmentation
train_transform = transforms.Compose(
    [transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))])

valid_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))])

In [ ]:
# loading data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

validset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=valid_transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# validation split
val_size = 0.1
num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(val_size * num_train))
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
# defining dataloaders
batch_size = 128
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          sampler = train_sampler, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          sampler = valid_sampler, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Network definitions

In [ ]:
arch = "resnetlike"

In [ ]:
# some hyperparameters for the network
epochs = 100
learning_rate = 3e-3
decay = 0.1
opt_milestones = [50, 75]

In [ ]:
# importing model to be used
model_file = __import__(f'models.{arch}', fromlist=[arch])
base_arch = getattr(model_file, arch)

In [ ]:
# moving net to GPU and summary of model architecture
net = base_arch()
net.to(device)
summary(net, input_size = (batch_size, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
resnetlike                               --                        --
├─Conv2d: 1-1                            [128, 32, 32, 32]         2,432
├─BatchNorm2d: 1-2                       [128, 32, 32, 32]         64
├─VanillaResBlock: 1-3                   [128, 32, 32, 32]         --
│    └─AutoConv2d_BN: 2-1                [128, 32, 32, 32]         --
│    │    └─Conv2d: 3-1                  [128, 32, 32, 32]         9,248
│    │    └─BatchNorm2d: 3-2             [128, 32, 32, 32]         64
│    └─AutoConv2d_BN: 2-2                [128, 32, 32, 32]         --
│    │    └─Conv2d: 3-3                  [128, 32, 32, 32]         9,248
│    │    └─BatchNorm2d: 3-4             [128, 32, 32, 32]         64
│    └─AutoConv2d_BN: 2-3                [128, 32, 32, 32]         --
│    │    └─Conv2d: 3-5                  [128, 32, 32, 32]         1,056
│    │    └─BatchNorm2d: 3-6             [128, 32, 32, 32]         64
├─V

In [ ]:
# defining loss, optimizer and lr scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr = learning_rate, weight_decay = decay)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, opt_milestones)

# Training

In [ ]:
def train(epoch):
    """training loop for one epoch"""
    net.train()

    running_loss = 0.0
    train_total = 0
    train_correct = 0

    for data in tqdm(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        running_loss += loss.item()
        predicted = torch.argmax(outputs.data, dim = 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
    scheduler.step() # only after every epoch

    # print statistics
    train_accuracy = 100 * (train_correct / train_total)
    print("Epoch", epoch + 1)
    print(f'    Loss of the network on the {train_total} training images: {running_loss}')
    print(f'    Accuracy of the network on the {train_total} training images: {train_accuracy}%')

In [ ]:
def validation():
    """validation set evaluation for one epoch"""
    valid_correct = 0
    valid_total = 0

    # since we're not training, we don't need to calculate the gradients
    with torch.no_grad():
        net.eval()

        for data in validloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            predicted = torch.argmax(outputs.data, dim = 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()

    valid_accuracy = 100 * (valid_correct / valid_total)
    print(f'    Accuracy of the network on the {valid_total} validation images: {valid_accuracy}%')

In [19]:
# training

for epoch in range(epochs):  # loop over the dataset multiple times
    train(epoch)
    validation()

print('Finished Training')


Epoch 1
    Loss of the network on the 45000 training images: 584.1680057048798
    Accuracy of the network on the 45000 training images: 37.6%
    Accuracy of the network on the 5000 validation images: 41.699999999999996%



Epoch 2
    Loss of the network on the 45000 training images: 433.79124414920807
    Accuracy of the network on the 45000 training images: 55.266666666666666%
    Accuracy of the network on the 5000 validation images: 49.5%



Epoch 3
    Loss of the network on the 45000 training images: 356.7426227927208
    Accuracy of the network on the 45000 training images: 63.855555555555554%
    Accuracy of the network on the 5000 validation images: 62.94%



Epoch 4
    Loss of the network on the 45000 training images: 309.5078405737877
    Accuracy of the network on the 45000 training images: 68.93777777777778%
    Accuracy of the network on the 5000 validation images: 62.239999999999995%



Epoch 5
    Loss of the network on the 45000 training images: 281.14050525426865
    Accuracy of the network on the 45000 training images: 71.96000000000001%
    Accuracy of the network on the 5000 validation images: 63.75999999999999%


...


Epoch 96
    Loss of the network on the 45000 training images: 30.49069169908762
    Accuracy of the network on the 45000 training images: 97.06222222222223%
    Accuracy of the network on the 5000 validation images: 91.25999999999999%



Epoch 97
    Loss of the network on the 45000 training images: 29.95830656029284
    Accuracy of the network on the 45000 training images: 97.19333333333333%
    Accuracy of the network on the 5000 validation images: 91.4%



Epoch 98
    Loss of the network on the 45000 training images: 30.843075893819332
    Accuracy of the network on the 45000 training images: 97.07777777777777%
    Accuracy of the network on the 5000 validation images: 91.28%



Epoch 99
    Loss of the network on the 45000 training images: 29.69056478701532
    Accuracy of the network on the 45000 training images: 97.11333333333333%
    Accuracy of the network on the 5000 validation images: 91.3%



Epoch 100
    Loss of the network on the 45000 training images: 30.240707842633128
    Accuracy of the network on the 45000 training images: 97.0911111111111%
    Accuracy of the network on the 5000 validation images: 91.3%
Finished Training


In [20]:
# saving model
model_save_name = f'cifar{arch}.pt'
path = f"/content/gdrive/My Drive/Deep Learning/pytorch-experiments/Vision/CIFAR-10/models/trained models/{model_save_name}" 
torch.save(net, path)

# Evaluation

In [21]:
# loading model
model_save_name = f'cifar{arch}.pt'
PATH = f"/content/gdrive/My Drive/Deep Learning/pytorch-experiments/Vision/CIFAR-10/models/trained models/{model_save_name}"
model = torch.load(PATH, map_location=torch.device(device))

In [22]:
# moving model to GPU
model.to(device);

In [23]:
# evaluating on test set
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct / total}')

Accuracy of the network on the 10000 test images: 91.01
